# Canary Deployment with Seldon and Istio

## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - A running Kubernetes cluster with kubectl authenticated
 - [seldon-core Python package](https://pypi.org/project/seldon-core/) (```pip install seldon-core>=0.2.6.1```)
 - [Helm client](https://helm.sh/)

### Creating a Kubernetes Cluster

Follow the [Kubernetes documentation to create a cluster](https://kubernetes.io/docs/setup/).

***This demo needs egress when running the load test to allow MNIST digits to be downloaded. If you want to run the load test then you will need to follow the docs on egress [here](https://istio.io/docs/tasks/traffic-management/egress/#calling-external-services-directly) if you run istio in a way that egress is blocked***

Once created ensure ```kubectl``` is authenticated against the running cluster.

## Setup

In [1]:
INGRESS_HOST=!kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
INGRESS_PORT=!kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.spec.ports[?(@.name=="http2")].port}'
ISTIO_GATEWAY=INGRESS_HOST[0]+":"+INGRESS_PORT[0]

Check the istio gateway address

In [2]:
ISTIO_GATEWAY

'10.202.115.191:80'

To view the istio traffic you can go to the istio grafana dashboard. In a separate terminal port-forward to it:

See their docs [here](https://istio.io/docs/tasks/telemetry/metrics/using-istio-dashboard/)

In [3]:
!kubectl apply -f mnist_v1.json

seldondeployment.machinelearning.seldon.io/mnist-classifier created


In [4]:
!kubectl rollout status deploy/mnist-deployment-sk-mnist-predictor-73d7608

deployment "mnist-deployment-sk-mnist-predictor-73d7608" successfully rolled out


## Start a Load Test

In [6]:
!helm install ../../../helm-charts/seldon-core-loadtesting --name loadtest  \
    --namespace seldon \
    --repo https://storage.googleapis.com/seldon-charts \
    --set locust.script=mnist_rest_locust.py \
    --set locust.host=http://{ISTIO_GATEWAY} \
    --set rest.pathPrefix=/seldon/seldon/mnist-classifier \
    --set oauth.enabled=false \
    --set oauth.key=oauth-key \
    --set oauth.secret=oauth-secret \
    --set locust.hatchRate=1 \
    --set locust.clients=1 \
    --set loadtest.sendFeedback=1 \
    --set locust.minWait=0 \
    --set locust.maxWait=0 \
    --set replicaCount=1 \
    --set data.size=784


Error: a release named loadtest already exists.
Run: helm ls --all loadtest; to check the status of the release
Or run: helm del --purge loadtest; to delete it


Now we will add a canary and split traffic 75% to 25% to it. This is done by adding a new predictor to the SeldonDeployment and specifying the traffic values.

In [7]:
!pygmentize mnist_v2.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "mnist-classifier"
    },
    "spec": {
        "annotations": {
            "project_name": "Mnist classification"
        },
        "name": "mnist-deployment",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/sk-example-mnist:0.2",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "sk-mnist-classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
                                    }
                                }
                            }
                    

In [12]:
!kubectl apply -f mnist_v2.json

seldondeployment.machinelearning.seldon.io/mnist-classifier configured


You should see traffic being split on the Istio service dashboard for the mnist-classifier.

![skpredictor](sk-predictor.png)



![tfpredictor](tf-predictor.png)

When you are happy the canary is ok you can promote to full traffic.

In [13]:
!kubectl apply -f mnist_v3.json

seldondeployment.machinelearning.seldon.io/mnist-classifier configured


You should now see all traffic transfer to the canary.

![sk-preditor2](sk-predictor2.png)

![tf-preditor2](tf-predictor2.png)